## Data

El equipo del startup a compartido los siguientes datos:

#### Redesign test data
- "treatment" - "yes" si el usuario vió la nueva versión de la página web, no en otro caso.
- "new_images" - "yes" si la página uso el nuevo conjunto de imagenes, no en otro caso.
- "converted" - 1 si el usuario se unio al sitio, 0 en otro caso.

El grupo de control son los usuarios con "no" en ambas columnas: la versión antigua con el viejo conjunto de imagenes.

In [41]:
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import chi2_contingency

df = pd.read_csv('./data/redesign.csv')
df.head()

,treatment,new_images,converted
0,yes,yes,0
1,yes,yes,0
2,yes,yes,0
3,yes,no,0
4,no,yes,0


## Preparación de Datos

In [42]:
df.value_counts()

treatment  new_images  converted
no         no          0            9037
           yes         0            8982
yes        yes         0            8970
           no          0            8906
                       1            1215
           yes         1            1151
no         yes         1            1139
           no          1            1084
dtype: int64

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40484 entries, 0 to 40483
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   treatment   40484 non-null  object
 1   new_images  40484 non-null  object
 2   converted   40484 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 949.0+ KB


In [44]:
df.isnull().any()

treatment     False
new_images    False
converted     False
dtype: bool

In [45]:
df.value_counts(df['treatment'])

treatment
no     20242
yes    20242
dtype: int64

In [46]:
df.value_counts(df['new_images'])

new_images
no     20242
yes    20242
dtype: int64

In [47]:
df.value_counts(df['converted'])

converted
0    35895
1     4589
dtype: int64

## Análisis Exploratorio.

### El diseño nuevo/viejo de la página web y las imagenes nuevas/viejas.

#### Radio de Conversión 

In [48]:
# Agrupar los datos por treatmen y new_images, calculamos la media del radio de conversión para cada grupo.
conversion_rates = df.groupby(['treatment', 'new_images'])['converted'].mean()

# Imprimir los radios de conversión para cada grupo
print(conversion_rates)

treatment  new_images
no         no            0.107104
           yes           0.112538
yes        no            0.120047
           yes           0.113724
Name: converted, dtype: float64


### ¿Pueden los incrementos observados ser explicador por el azar?

Para que podamos hacer esta observación hacemos el siguiente A/B test (chi square)

In [50]:
# Crear una tabla de contingencia que cuente cada una de la combinaciones posibles.
contingency_table = pd.crosstab(df['treatment'], df['new_images'], values=df['converted'], aggfunc='sum')

# Hacer un chi-squared test
chi2, p_value, _, _ = chi2_contingency(contingency_table)

# Establecer el nivel de significancia
alpha = 0.05

# Checar si el p-value is menos que el nivel de significancia
if p_value < alpha:
    print("Los incrementos observados en los radios de conversión son estadisticamente significantes.")
else:
    print("Los incrementos observados en los radios de conversión no son estadisticamente significantes")

Los incrementos observados en los radios de conversión no son estadisticamente significantes


### ¿Cuál es la versión de la pagina que es más recomendada usar?

Basados en el A/B test, podemos ver que los aumentos en los radios de conversión no tienen significancia estadistica, es decir, no podemos concluir que alguna de las comvinaciones sea la más adeacuada para usar.
Al no poder definir dicha combinaciones en base al A/B test, podemos simplemente decir que ya sería lo "mejor" usar la combinacion que mayor radio de converción tuvo, **treatment = Yes** and **new_images = No**